In [1]:
!pip install vk_api

In [2]:
import vk_api

In [ ]:
posts = vk.wall.get(owner_id=-20629724, count=100)['items'][10]
posts

In [ ]:
import csv
import pandas as pd

def scrape_wall_data(token, owner_id, count=10):
    try: # блок try-except для обработки возможных ошибок при взаимодействии с API ВКонтакте.
        vk_session = vk_api.VkApi(token=token) #выполянем запрос по токену
        vk = vk_session.get_api() #получаем объект API, с помощью которого будут выполняться запросы к серверам ВКонтакте.
       
        # Метод wall.get
        posts = vk.wall.get(owner_id=owner_id, count=count)['items'] # записываем записи со стены сообщества в переменную posts

        # Хранилище для данных
        scraped_data = [] # пустой список для добавления данных с каждой записи при иритации 

        # Парсинг
        for post in posts:
            post_data = {
                'post_id': post['id'],
                'text': post['text'],
                'likes': post['likes'],
            }
            scraped_data.append(post_data)

        return scraped_data

    except vk_api.exceptions.ApiError as e:
        print(f"Error accessing VK API: {e}")
        return []

In [ ]:
def save_to_csv(data, file_name='vk_wall_posts.csv'): #сохраним наши данные в файл формата csv
    with open(file_name, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['post_id', 'text','likes'])
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def stats(data):
    df = pd.DataFrame(data)
    
    df['like_count'] = df['likes'].apply(lambda x: x['count']) # посмотрим на количество лайков под постами 
    df.drop('likes', axis=1, inplace=True)
    
    # визализация статистики по количеству лайков
    plt.bar(df['post_id'], df['like_count'])
    plt.xlabel('Post ID')
    plt.ylabel('Number of Likes')
    plt.title('Likes per Post')
    plt.show()
    


    print(f'Посмортим на общую информацию: \n {df.describe()}')
    print('\n')
    print(df.info())
    print('\n')
    print(f'Статистика по первым пяти постам\n{df.head()}')
    pass

In [ ]:
def main():
    # Введите ваш токен VK API

    # Введите ID сообщества
    owner_id = -299

    # Введите количество публикаций для скрейпинга
    num_posts = 20

    wall_data = scrape_wall_data(token, owner_id, num_posts)
    print('Посмотрим на полученные данные')
    print(wall_data)
    print('\n')
    if wall_data:
        save_to_csv(wall_data)

        stats(wall_data)
    else:
        print("No data")

if __name__ == "__main__":
    main()